In [1]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple

In [24]:
Genome = List[int]
Population = List[Genome]
Populate_function = Callable[[], Population]
Fitness_function = Callable[[Genome], int]
Selection_function = Callable[[Population, Fitness_function], Tuple[Genome, Genome]]
Crossover_function = Callable[[Genome, Genome], Tuple[Genome, Genome]]
Mutation_function = Callable[[Genome], Genome]
Printer_function = Callable[[Population, int, Fitness_function], None]

In [4]:
def generate_genome(length: int) -> Genome:
    return choices([0,1], k = length)

In [8]:
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

In [9]:
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError('Their lengths must be the same')
     
    length = len(a)
    if length == 2:
        return a,b
    
    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

In [10]:
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

In [15]:
def popultaion_fitness(genome: Genome, fitness_func: Fitness_function) -> int:
    return sum(fitness_func(genome) for genome in Popultaion)

In [18]:
def selection_pair(population: Population, fitness_func: Fitness_function) -> Population:
    return sample(
    population = generate_weighted_distribution(population, fitness_func), k = 2)

In [19]:
def generate_weighted_distribution(population: Population, fitness_func: Fitness_function) -> Population:
    result = []
    
    for gene in population:
        result += [gene] * int(fitness_func(gene) + 1)
        
    return result

In [20]:
def sort_population(population: Population, fitness_func: Fitness_function) -> Population:
    return sorted(population, key = fitness_func, reverse=True)

In [21]:
def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

In [23]:
def print_stats(population: Population, generation_id: int, fitness_func: Fitness_function):
    
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print("Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


In [27]:
def run_evolution(
                populate_func: Populate_function,
                fitness_func: Fitness_function,
                fitness_limit: int,
                selection_func: Selection_function = selection_pair,
                crossover_func: Crossover_function = single_point_crossover,
                mutation_func: Mutation_function = mutation,
                generation_limit: int=100,
                printer: Optional[Printer_function] = None) -> Tuple[Population, int]:
    
    population = populate_func()
    
    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)
        
        if printer is not None:
            printer(population, i, fitness_func)
            
        if fitness_func(population[0]) >= fitness_limit:
            break
        
        next_generation = population[0:2]
        
        for j in range(int(len(population)/2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_A, offspring_B = crossover_func(parents[0], parents[1])
            offspring_A = mutation_func(offspring_A)
            offspring_B = mutation_func(offspring_B)
            
            next_generation += [offspring_A, ofsspring_B]
            
        population = next_generation
        
    return population, i 